In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from time import time
from keras.models import load_model

In [2]:
#gpu memory growth fix
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# Loading data
train_data = pd.read_csv('E:\Python\data\Titanic\prs_train.csv')
train_data
test_data = pd.read_csv('E:\Python\data\Titanic\prs_test.csv')
test_data
# train and test split
train_X, train_y = train_data.values[:, 2:], train_data.values[:, 1]
test_X = test_data.values[:, 1:]
# One hot encoding
enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
train_X_enc = enc.fit_transform(train_X)
test_X_enc = enc.transform(test_X)
# change type of data for nn to work
train_X_enc=np.asarray(train_X_enc).astype(np.float32)
train_y_enc=np.asarray(train_y).astype(np.float32)
test_X_enc=np.asarray(test_X_enc).astype(np.float32)

In [4]:
from sklearn.model_selection import train_test_split, KFold
import scipy
first_col = True
cross_fold = KFold(n_splits = 7, shuffle=True)
for train_index, test_index in cross_fold.split(train_X_enc):
    validation_X, validation_y = train_X_enc[test_index], train_y_enc[test_index]
    train_X, train_y = train_X_enc[train_index], train_y_enc[train_index]
    
    # determine the number of input features
    n_features = train_X_enc.shape[1]
    # define model
    model = Sequential()
    model.add(layers.Dense(512, kernel_initializer='he_normal', input_shape=(n_features,)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(512, kernel_initializer='he_normal'))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Activation('elu'))
#     model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2, activation='softmax'))
    # compile the model
    model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
    #callbacks
    # simple early stopping
    es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001, verbose=1, patience=30)
    mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

    tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
    
    rlrop = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=5, verbose=1)
    
    # fit model and save the best
    history = model.fit(x=train_X, y=train_y, 
#                         steps_per_epoch=steps_per_epoch, 
                        batch_size=8, 
                        epochs=300, 
                        validation_data=(validation_X, validation_y), 
#                         validation_steps=validation_steps, 
                        shuffle=True, 
                        callbacks=[tensorboard, es, mc, rlrop]
                       )
    saved_model = load_model('best_model.h5')
    
    probs = saved_model.predict(test_X_enc)
    predict = probs.argmax(axis=1)
    if first_col:
        pr_values = np.array(predict, ndmin=2)
        pr_values = np.transpose(pr_values)
        first_col = False
    else:
        pr_values = np.insert(pr_values, -1, predict, axis=1)
pr_values= scipy.stats.mode(pr_values, axis=1)

Epoch 1/300
 1/96 [..............................] - ETA: 0s - loss: 0.8420 - accuracy: 0.5000WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
91/96 [===========================>..] - ETA: 0s - loss: 0.7811 - accuracy: 0.7129
Epoch 00001: val_accuracy improved from -inf to 0.80315, saving model to best_model.h5
96/96 [==============================] - 2s 19ms/step - loss: 0.7707 - accuracy: 0.7192 - val_loss: 0.4852 - val_accuracy: 0.8031
Epoch 2/300
91/96 [===========================>..] - ETA: 0s - loss: 0.5929 - accuracy: 0.7582
Epoch 00002: val_accuracy did not improve from 0.80315
96/96 [==============================] - 1s 9ms/step - loss: 0.5847 - accuracy: 0.7612 - val_loss: 0.4934 - val_accuracy: 0.8031
Epoch 3/300
88/96 [==========================

Epoch 28/300
90/96 [===========================>..] - ETA: 0s - loss: 0.2313 - accuracy: 0.9069
Epoch 00028: val_accuracy did not improve from 0.81890
96/96 [==============================] - 1s 7ms/step - loss: 0.2285 - accuracy: 0.9081 - val_loss: 0.8151 - val_accuracy: 0.7953
Epoch 29/300
89/96 [==========================>...] - ETA: 0s - loss: 0.2583 - accuracy: 0.8989
Epoch 00029: val_accuracy did not improve from 0.81890
96/96 [==============================] - 1s 7ms/step - loss: 0.2575 - accuracy: 0.9003 - val_loss: 0.7765 - val_accuracy: 0.7874
Epoch 30/300
95/96 [============================>.] - ETA: 0s - loss: 0.2101 - accuracy: 0.9211
Epoch 00030: val_accuracy did not improve from 0.81890
96/96 [==============================] - 1s 9ms/step - loss: 0.2102 - accuracy: 0.9213 - val_loss: 0.8075 - val_accuracy: 0.7953
Epoch 31/300
94/96 [============================>.] - ETA: 0s - loss: 0.1947 - accuracy: 0.9229
Epoch 00031: val_accuracy did not improve from 0.81890

Epoch 00

Epoch 19/300
96/96 [==============================] - ETA: 0s - loss: 0.2823 - accuracy: 0.8898
Epoch 00019: val_accuracy did not improve from 0.82677
96/96 [==============================] - 1s 6ms/step - loss: 0.2823 - accuracy: 0.8898 - val_loss: 0.5773 - val_accuracy: 0.8031
Epoch 20/300
88/96 [==========================>...] - ETA: 0s - loss: 0.2505 - accuracy: 0.9020
Epoch 00020: val_accuracy did not improve from 0.82677
96/96 [==============================] - 1s 6ms/step - loss: 0.2441 - accuracy: 0.9055 - val_loss: 0.5831 - val_accuracy: 0.7953
Epoch 21/300
93/96 [============================>.] - ETA: 0s - loss: 0.2780 - accuracy: 0.9032
Epoch 00021: val_accuracy did not improve from 0.82677
96/96 [==============================] - 1s 7ms/step - loss: 0.2728 - accuracy: 0.9042 - val_loss: 0.6430 - val_accuracy: 0.7559
Epoch 22/300
95/96 [============================>.] - ETA: 0s - loss: 0.2770 - accuracy: 0.8974
Epoch 00022: val_accuracy did not improve from 0.82677
96/96 [==

Epoch 2/300
91/96 [===========================>..] - ETA: 0s - loss: 0.5035 - accuracy: 0.7747
Epoch 00002: val_accuracy improved from 0.77953 to 0.82677, saving model to best_model.h5
96/96 [==============================] - 1s 7ms/step - loss: 0.5130 - accuracy: 0.7730 - val_loss: 0.4361 - val_accuracy: 0.8268
Epoch 3/300
90/96 [===========================>..] - ETA: 0s - loss: 0.4726 - accuracy: 0.8111
Epoch 00003: val_accuracy improved from 0.82677 to 0.83465, saving model to best_model.h5
96/96 [==============================] - 1s 7ms/step - loss: 0.4889 - accuracy: 0.8071 - val_loss: 0.3910 - val_accuracy: 0.8346
Epoch 4/300
87/96 [==========================>...] - ETA: 0s - loss: 0.4478 - accuracy: 0.8118
Epoch 00004: val_accuracy did not improve from 0.83465
96/96 [==============================] - 1s 6ms/step - loss: 0.4503 - accuracy: 0.8123 - val_loss: 0.4113 - val_accuracy: 0.8346
Epoch 5/300
95/96 [============================>.] - ETA: 0s - loss: 0.4225 - accuracy: 0.814

Epoch 31/300
96/96 [==============================] - ETA: 0s - loss: 0.2689 - accuracy: 0.8806
Epoch 00031: val_accuracy did not improve from 0.84252
96/96 [==============================] - 1s 9ms/step - loss: 0.2689 - accuracy: 0.8806 - val_loss: 0.5409 - val_accuracy: 0.8031
Epoch 32/300
91/96 [===========================>..] - ETA: 0s - loss: 0.2122 - accuracy: 0.9162
Epoch 00032: val_accuracy did not improve from 0.84252
96/96 [==============================] - 1s 8ms/step - loss: 0.2073 - accuracy: 0.9186 - val_loss: 0.5628 - val_accuracy: 0.7874
Epoch 33/300
95/96 [============================>.] - ETA: 0s - loss: 0.2148 - accuracy: 0.9105
Epoch 00033: val_accuracy did not improve from 0.84252
96/96 [==============================] - 1s 8ms/step - loss: 0.2149 - accuracy: 0.9108 - val_loss: 0.5446 - val_accuracy: 0.8189
Epoch 34/300
96/96 [==============================] - ETA: 0s - loss: 0.2257 - accuracy: 0.9147
Epoch 00034: val_accuracy did not improve from 0.84252
96/96 [==

...

Epoch 16/300
93/96 [============================>.] - ETA: 0s - loss: 0.2766 - accuracy: 0.8858
Epoch 00016: val_accuracy did not improve from 0.83465
96/96 [==============================] - 1s 7ms/step - loss: 0.2798 - accuracy: 0.8871 - val_loss: 0.6816 - val_accuracy: 0.7717
Epoch 17/300
94/96 [============================>.] - ETA: 0s - loss: 0.2499 - accuracy: 0.9043
Epoch 00017: val_accuracy did not improve from 0.83465
96/96 [==============================] - 1s 7ms/step - loss: 0.2489 - accuracy: 0.9042 - val_loss: 0.7136 - val_accuracy: 0.7717
Epoch 18/300
90/96 [===========================>..] - ETA: 0s - loss: 0.2316 - accuracy: 0.8986
Epoch 00018: val_accuracy did not improve from 0.83465
96/96 [==============================] - 1s 7ms/step - loss: 0.2382 - accuracy: 0.8976 - val_loss: 0.7900 - val_accuracy: 0.7638
Epoch 19/300
92/96 [===========================>..] - ETA: 0s - loss: 0.2747 - accuracy: 0.8886
Epoch 00019: val_accuracy did not improve from 0.83465

Epoch 00

91/96 [===========================>..] - ETA: 0s - loss: 0.3405 - accuracy: 0.8599
Epoch 00010: val_accuracy did not improve from 0.81890
96/96 [==============================] - 1s 8ms/step - loss: 0.3384 - accuracy: 0.8570 - val_loss: 0.6183 - val_accuracy: 0.7717
Epoch 11/300
93/96 [============================>.] - ETA: 0s - loss: 0.3277 - accuracy: 0.8562
Epoch 00011: val_accuracy did not improve from 0.81890
96/96 [==============================] - 1s 8ms/step - loss: 0.3227 - accuracy: 0.8583 - val_loss: 0.6096 - val_accuracy: 0.7480
Epoch 12/300
94/96 [============================>.] - ETA: 0s - loss: 0.2978 - accuracy: 0.8803
Epoch 00012: val_accuracy did not improve from 0.81890
96/96 [==============================] - 1s 10ms/step - loss: 0.2965 - accuracy: 0.8806 - val_loss: 0.6083 - val_accuracy: 0.7795
Epoch 13/300
95/96 [============================>.] - ETA: 0s - loss: 0.2906 - accuracy: 0.8921
Epoch 00013: val_accuracy did not improve from 0.81890
96/96 [==============

Epoch 6/300
93/96 [============================>.] - ETA: 0s - loss: 0.3340 - accuracy: 0.8656
Epoch 00006: val_accuracy improved from 0.79528 to 0.81102, saving model to best_model.h5
96/96 [==============================] - 1s 8ms/step - loss: 0.3347 - accuracy: 0.8661 - val_loss: 0.5290 - val_accuracy: 0.8110
Epoch 7/300
89/96 [==========================>...] - ETA: 0s - loss: 0.3121 - accuracy: 0.8750
Epoch 00007: val_accuracy improved from 0.81102 to 0.82677, saving model to best_model.h5
96/96 [==============================] - 1s 9ms/step - loss: 0.3207 - accuracy: 0.8701 - val_loss: 0.4706 - val_accuracy: 0.8268
Epoch 8/300
96/96 [==============================] - ETA: 0s - loss: 0.3361 - accuracy: 0.8596
Epoch 00008: val_accuracy did not improve from 0.82677
96/96 [==============================] - 1s 7ms/step - loss: 0.3361 - accuracy: 0.8596 - val_loss: 0.5925 - val_accuracy: 0.8110
Epoch 9/300
95/96 [============================>.] - ETA: 0s - loss: 0.3791 - accuracy: 0.847

Epoch 35/300
89/96 [==========================>...] - ETA: 0s - loss: 0.1946 - accuracy: 0.9242
Epoch 00035: val_accuracy did not improve from 0.85039
96/96 [==============================] - 1s 6ms/step - loss: 0.2132 - accuracy: 0.9160 - val_loss: 0.5393 - val_accuracy: 0.8268
Epoch 36/300
96/96 [==============================] - ETA: 0s - loss: 0.2013 - accuracy: 0.9278
Epoch 00036: val_accuracy did not improve from 0.85039
96/96 [==============================] - 1s 7ms/step - loss: 0.2013 - accuracy: 0.9278 - val_loss: 0.6216 - val_accuracy: 0.8189
Epoch 37/300
96/96 [==============================] - ETA: 0s - loss: 0.2533 - accuracy: 0.9042
Epoch 00037: val_accuracy did not improve from 0.85039
96/96 [==============================] - 1s 6ms/step - loss: 0.2533 - accuracy: 0.9042 - val_loss: 0.5940 - val_accuracy: 0.7874
Epoch 38/300
94/96 [============================>.] - ETA: 0s - loss: 0.2065 - accuracy: 0.9176
Epoch 00038: val_accuracy did not improve from 0.85039
96/96 [==

Epoch 00062: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
96/96 [==============================] - 1s 10ms/step - loss: 0.1650 - accuracy: 0.9344 - val_loss: 0.5916 - val_accuracy: 0.8425
Epoch 63/300
91/96 [===========================>..] - ETA: 0s - loss: 0.1396 - accuracy: 0.9478
Epoch 00063: val_accuracy did not improve from 0.85827
96/96 [==============================] - 1s 8ms/step - loss: 0.1401 - accuracy: 0.9475 - val_loss: 0.5929 - val_accuracy: 0.8346
Epoch 64/300
92/96 [===========================>..] - ETA: 0s - loss: 0.1767 - accuracy: 0.9293
Epoch 00064: val_accuracy did not improve from 0.85827
96/96 [==============================] - 1s 9ms/step - loss: 0.1763 - accuracy: 0.9304 - val_loss: 0.5931 - val_accuracy: 0.8425
Epoch 65/300
96/96 [==============================] - ETA: 0s - loss: 0.1745 - accuracy: 0.9291
Epoch 00065: val_accuracy did not improve from 0.85827
96/96 [==============================] - 1s 7ms/step - loss: 0.1745 - accuracy:

In [5]:
z = zip(test_data.iloc[:, 0], [x[-1] for x in pr_values[0]])
data3 = pd.DataFrame(z, columns=['PassengerId', 'Survived'])
data3

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [6]:
# saving results
data3.to_csv('E:/Python/data/Titanic/gender_submission19.csv', index=False)